# Business Understanding

## Background
My company is looking to get into movie creation using their newly created movie studio. 

## Business Goals
The primary focus of this data science project is to analyze and assess features that contribute to a movie's return on investment in order to make an informed decision regarding what features of movie creation my company should invest in.

## Business Success Criteria
The success of this project will be measured by providing three well-supported recommendations on the most cost efficient movie features (actors, directots, genre, marketing cost, movie rating (G, PG, PG-13, etc.)) to invest in. For this project, the most "cost efficient" features are measured by their return on investment which is defined as 100% times the total revenue divided by the initial investment of the feature.

# Data Understanding

Data on movies is collected by a variety of different sources. For this project, I used data from the following sources:
- Box Office Mojo's "Movie Gross" dataset
    - This dataset includes 5 features and 3387 observations. Each entry in the dataset represents a different movie title. Additional information is included about each entry, such as the production studio, the domestic gross box office, the foreign gross box office, and the release year.
    
    
- Rotten Tomatoes "Movie Info" dataset
    - This dataset includes 12 features and 1560 observations/entries where each entry in the dataset represents a different movie. Even though the movie title is not included, each entry has an associated unique ID. Additional information is included about each entry, such as a synopsis, the movie rating, genre(s), director, writer, and runtime.
    
    
- Rotten Tomatoes "Movie Reviews" dataset
    - This dataset includes 8 features and 54432 observations/entries, where each entry in the dataset represents a different review left by a movie critic. Each entry contains an ID which refers to the movie the review is about. In addition to the review, other information for each entry includes a rating (out of 5), the critic's name, the publisher, and the date the review was left. 


- The Movie DB's "Movies" dataset
    - This dataset includes 10 features and 26517 observations. Each entry in the dataset represents a different movie. For each entry information is included about the movie's genre, original language, popularity score, and number of votes. 
    
    
- The Numbers' "movie_budgets" dataset
    - This dataset includes 6 features and 5782 observations. Each entry in the dataset represents a different movie. For each entry, information is included about the movie's release data, production budget, domestic gross box office, and worldwide gross box office.
    
    
- IMDB's film database
    - This database includes 8 tables. Its entity-related diagram (ERD) is shown below.  

## Data Preparation

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
budgets = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [3]:
budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [4]:
# defining columns to format

budget_cols = ['production_budget', 'domestic_gross', 'worldwide_gross']


In [9]:
# formatting budget columns by removing the $

for col in budget_cols:
     budgets[col] = budgets[col].str.replace('$', '')
        
        
# formatting budget columns by removing commas

for col in budget_cols:
     budgets[col] = budgets[col].str.replace(',', '')

In [11]:
# changing dtype of budget and gross columns to int

for col in budget_cols:
     budgets[col] = budgets[col].astype(float)

In [12]:
#converting dollar amounts to amounts that are easier to read

for col in budget_cols:
    budgets[col] = round(budgets[col]/1000000, 3)

In [13]:
# renaming columns to include dollar sign
budgets.rename(columns={'production_budget': 'production_budget (millions of $)', 'domestic_gross': 'domestic_gross (millions of $)', 'worldwide_gross' : 'worldwide_gross (millions of $)'}, inplace=True)


In [14]:
budgets.head()

,id,release_date,movie,production_budget (millions of $),domestic_gross (millions of $),worldwide_gross (millions of $)
0,1,"Dec 18, 2009",Avatar,425.0,760.508,2776.345
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410.6,241.064,1045.664
2,3,"Jun 7, 2019",Dark Phoenix,350.0,42.762,149.762
3,4,"May 1, 2015",Avengers: Age of Ultron,330.6,459.006,1403.014
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317.0,620.181,1316.722


# Exploratory Data Analysis

# Conclusion
 
## Limitations

## Recommendations

## Next Steps

# Resources

- got error when trying to read in reviews.tsv dataset: "UnicodeDecodeError: 'utf8' codec can't decode byte 0xa5 in position 0: invalid start byte". This resource helped: https://stackoverflow.com/questions/22216076/unicodedecodeerror-utf8-codec-cant-decode-byte-0xa5-in-position-0-invalid-s

- Had trouble reading zipped data with pandas. This helped me with reading in the csv and tsv data: https://stackoverflow.com/questions/18885175/read-a-zipped-file-as-a-pandas-dataframe

- for joining datasets: https://stackoverflow.com/questions/53645882/pandas-merging-101

- the movie_info dataset had the word 'minutes' in each entry for runtime. Just wanted the numbers. this was helpful:
https://saturncloud.io/blog/how-to-remove-characters-from-a-pandas-column-a-data-scientists-guide/